# <span style="color:LightSlateGray;">Data Science Project Assessment of Database Performance Degradation</span>
_CAS ADS 2019/2020 University of Berne_<br/>
_Author: Marco Bassi_, September 20, 2019

## Introduction

### Situation
The SBB project KiHub operates the CUS platform, a datahub for the real-time data of the Swiss public transport. One of the core components it the CUS DB, an Oracle RDBMS. There is the **CUS DB Prod** for the productive environment, and the **CUS DB Inte** for the release test environment. Each database runs on its own, dedicated Oracle Data Appliance (ODA). Each database consists two database instances, running in cluster mode (Oracle Real Application Cluster).

Upon an upgrade of its grid infrastructure, the performance of the CUS DB Inte deteriorated to a degree, where it wouldn't meet anymore its purpose for the release test. The performance issues manifested themselves mainly in a massive degradation of the cluster-related wait times. 

In the beginning of August, a number of patches was applied to improve the performance. The goal of this analysis is to evaluate the performance of the CUS DB Inte, by compairing it with the CUS DB Prod. For this purpose, some selected system statistics are gathered from the _Oracle dynamic performance view_ DBA_HIST_SYSSTAT. This view contains hourly snapshots of historicised system statistics.
The statistics are running sums, with their values beeing reset upon system restart.

### Data Analysis
The analysis is made of three parts:
1. Part 1 contains the data cleansing.
2. Part 2 analyses the system load of the two databases. Similar system load is a major precondition for drawing conclusions of the performance of the CUS DB Inte, when compaired to CUS DB Prod.
3. Part 3 specifically analyses statistics related to the Global Cache (GC). GC is involved when one cluster database instance requires data currently held in the Local Cache (LC) of the other cluster database instance. The handling of this situation is ruled by a sophisticated protocol, which however is out of scope of this analysis.

## <span style="color:LightSlateGray;">Part 1 &mdash; Data Cleansing</span>

### Set the environment

In [1]:
library(data.table)
setwd('../../data')
getwd()

[1] "C:/Users/ue85374/data/CAS-ADS/Git.repos/CAS-Applied-Data-Science/Data-Science-Project/project.1/statistiken.2019-08-22T0000-bis-2019-08-28T1000/data"

### Read the data

In [2]:
daten.inte <- read.csv2(file="dba_hist_sysstat.inte.dsv", sep=";", dec=".", stringsAsFactors=F)
str(daten.inte)

'data.frame':	232897 obs. of  6 variables:
 $ BEGIN_INTERVAL_TIME: chr  "2019-08-22 00:00:05" "2019-08-22 00:00:05" "2019-08-22 00:00:05" "2019-08-22 00:00:05" ...
 $ END_INTERVAL_TIME  : chr  "2019-08-22 01:00:22" "2019-08-22 01:00:22" "2019-08-22 01:00:22" "2019-08-22 01:00:22" ...
 $ SNAP_ID            : int  20070 20070 20070 20070 20070 20070 20070 20070 20070 20070 ...
 $ INSTANCE_NUMBER    : int  2 2 2 2 2 2 2 2 2 2 ...
 $ STAT_NAME          : chr  "active txn count during cleanout" "ADG parselock X get attempts" "ADG parselock X get successes" "application wait time" ...
 $ VALUE              : num  94067174 0 0 24287453 0 ...


In [3]:
daten.prod <- read.csv2(file="dba_hist_sysstat.prod.dsv", sep=";", dec=".", stringsAsFactors=F)
str(daten.prod)

'data.frame':	239008 obs. of  6 variables:
 $ BEGIN_INTERVAL_TIME: chr  "2019-08-22 00:00:14" "2019-08-22 00:00:14" "2019-08-22 00:00:14" "2019-08-22 00:00:14" ...
 $ END_INTERVAL_TIME  : chr  "2019-08-22 01:00:16" "2019-08-22 01:00:16" "2019-08-22 01:00:16" "2019-08-22 01:00:16" ...
 $ SNAP_ID            : int  35567 35567 35567 35567 35567 35567 35567 35567 35567 35567 ...
 $ INSTANCE_NUMBER    : int  1 1 1 1 1 1 1 1 1 1 ...
 $ STAT_NAME          : chr  "active txn count during cleanout" "ADG parselock X get attempts" "ADG parselock X get successes" "application wait time" ...
 $ VALUE              : num  5.11e+08 0.00 0.00 8.39e+07 1.19e+02 ...


In [4]:
## Use data.table library
dt.inte <- data.table(daten.inte)
dt.prod <- data.table(daten.prod)
head(dt.prod, 5)

BEGIN_INTERVAL_TIME,END_INTERVAL_TIME,SNAP_ID,INSTANCE_NUMBER,STAT_NAME,VALUE
2019-08-22 00:00:14,2019-08-22 01:00:16,35567,1,active txn count during cleanout,511403667
2019-08-22 00:00:14,2019-08-22 01:00:16,35567,1,ADG parselock X get attempts,0
2019-08-22 00:00:14,2019-08-22 01:00:16,35567,1,ADG parselock X get successes,0
2019-08-22 00:00:14,2019-08-22 01:00:16,35567,1,application wait time,83919815
2019-08-22 00:00:14,2019-08-22 01:00:16,35567,1,auto extends on undo tablespace,119


### As the statistics are running sums, they have to be "nulled" with respect to the oldest snapshot common to the two instances of a cluster.

In [5]:
# find the minimum snaphost ID for the CUS DB Inte data
minsnap.inte <- dt.inte[, min(SNAP_ID), by=INSTANCE_NUMBER]

# select the minimum snapshot data
mindt.inte <- merge(dt.inte, minsnap.inte, by.x=c("INSTANCE_NUMBER", "SNAP_ID"), by.y=c("INSTANCE_NUMBER", "V1"), all=F)
dtNulled.inte <- merge(dt.inte, mindt.inte[, list(INSTANCE_NUMBER, STAT_NAME, VALUE)] , by=c("INSTANCE_NUMBER", "STAT_NAME"), suffixes=c("", 0),all.x = T)

# Check if there are statisctics having value NA
sum(is.na(dtNulled.inte[, c('VALUE', 'VALUE0')]))  # must be 0

# Compute the nulled value, i.e. the statics value of all snapshots and statistics wrt. the minimum snapshot data
dtNulled.inte[, offsetValue := VALUE - VALUE0]
str(dtNulled.inte)

[1] 0

Classes 'data.table' and 'data.frame':	232897 obs. of  8 variables:
 $ INSTANCE_NUMBER    : int  1 1 1 1 1 1 1 1 1 1 ...
 $ STAT_NAME          : chr  "ADG parselock X get attempts" "ADG parselock X get attempts" "ADG parselock X get attempts" "ADG parselock X get attempts" ...
 $ BEGIN_INTERVAL_TIME: chr  "2019-08-22 00:00:05" "2019-08-22 01:00:22" "2019-08-22 02:00:46" "2019-08-22 03:00:40" ...
 $ END_INTERVAL_TIME  : chr  "2019-08-22 01:00:22" "2019-08-22 02:00:46" "2019-08-22 03:00:40" "2019-08-22 04:00:21" ...
 $ SNAP_ID            : int  20070 20071 20072 20073 20074 20075 20076 20077 20078 20079 ...
 $ VALUE              : num  0 0 0 0 0 0 0 0 0 0 ...
 $ VALUE0             : num  0 0 0 0 0 0 0 0 0 0 ...
 $ offsetValue        : num  0 0 0 0 0 0 0 0 0 0 ...
 - attr(*, ".internal.selfref")=<externalptr> 
 - attr(*, "sorted")= chr  "INSTANCE_NUMBER" "STAT_NAME"


In [6]:
# same procedure for CUS DB Prod data
minsnap.prod <- dt.prod[, min(SNAP_ID), by=INSTANCE_NUMBER]
mindt.prod <- merge(dt.prod, minsnap.prod, by.x=c("INSTANCE_NUMBER", "SNAP_ID"), by.y=c("INSTANCE_NUMBER", "V1"), all=F)
dtNulled.prod <- merge(dt.prod, mindt.prod[, list(INSTANCE_NUMBER, STAT_NAME, VALUE)] , by=c("INSTANCE_NUMBER", "STAT_NAME"), suffixes=c("", 0),all.x = T)

sum(is.na(dtNulled.prod[, c('VALUE', 'VALUE0')]))  # must be 0
dtNulled.prod[, offsetValue := VALUE - VALUE0]
str(dtNulled.prod)

[1] 0

Classes 'data.table' and 'data.frame':	239008 obs. of  8 variables:
 $ INSTANCE_NUMBER    : int  1 1 1 1 1 1 1 1 1 1 ...
 $ STAT_NAME          : chr  "ADG parselock X get attempts" "ADG parselock X get attempts" "ADG parselock X get attempts" "ADG parselock X get attempts" ...
 $ BEGIN_INTERVAL_TIME: chr  "2019-08-22 00:00:14" "2019-08-22 01:00:16" "2019-08-22 02:00:01" "2019-08-22 03:00:07" ...
 $ END_INTERVAL_TIME  : chr  "2019-08-22 01:00:16" "2019-08-22 02:00:01" "2019-08-22 03:00:07" "2019-08-22 04:00:20" ...
 $ SNAP_ID            : int  35567 35568 35569 35570 35571 35572 35573 35574 35575 35576 ...
 $ VALUE              : num  0 0 0 0 0 0 0 0 0 0 ...
 $ VALUE0             : num  0 0 0 0 0 0 0 0 0 0 ...
 $ offsetValue        : num  0 0 0 0 0 0 0 0 0 0 ...
 - attr(*, ".internal.selfref")=<externalptr> 
 - attr(*, "sorted")= chr  "INSTANCE_NUMBER" "STAT_NAME"


In [7]:
# Compute datetime values from character strings
dtNulled.inte[, beginInterval := as.POSIXct(BEGIN_INTERVAL_TIME, format="%Y-%m-%d %H:%M:%S")]
dtNulled.inte[, endInterval := as.POSIXct(END_INTERVAL_TIME, format="%Y-%m-%d %H:%M:%S")]
dtNulled.inte[, snapHour := as.POSIXct(round(beginInterval, units="hours"))]

dtNulled.prod[, beginInterval := as.POSIXct(BEGIN_INTERVAL_TIME, format="%Y-%m-%d %H:%M:%S")]
dtNulled.prod[, endInterval := as.POSIXct(END_INTERVAL_TIME, format="%Y-%m-%d %H:%M:%S")]
dtNulled.prod[, snapHour := as.POSIXct(round(beginInterval, units="hours"))]
str(dtNulled.prod)

Classes 'data.table' and 'data.frame':	239008 obs. of  11 variables:
 $ INSTANCE_NUMBER    : int  1 1 1 1 1 1 1 1 1 1 ...
 $ STAT_NAME          : chr  "ADG parselock X get attempts" "ADG parselock X get attempts" "ADG parselock X get attempts" "ADG parselock X get attempts" ...
 $ BEGIN_INTERVAL_TIME: chr  "2019-08-22 00:00:14" "2019-08-22 01:00:16" "2019-08-22 02:00:01" "2019-08-22 03:00:07" ...
 $ END_INTERVAL_TIME  : chr  "2019-08-22 01:00:16" "2019-08-22 02:00:01" "2019-08-22 03:00:07" "2019-08-22 04:00:20" ...
 $ SNAP_ID            : int  35567 35568 35569 35570 35571 35572 35573 35574 35575 35576 ...
 $ VALUE              : num  0 0 0 0 0 0 0 0 0 0 ...
 $ VALUE0             : num  0 0 0 0 0 0 0 0 0 0 ...
 $ offsetValue        : num  0 0 0 0 0 0 0 0 0 0 ...
 $ beginInterval      : POSIXct, format: "2019-08-22 00:00:14" "2019-08-22 01:00:16" ...
 $ endInterval        : POSIXct, format: "2019-08-22 01:00:16" "2019-08-22 02:00:01" ...
 $ snapHour           : POSIXct, format: "2019-08

**N.B.** The field _snapHour_ is the time the hourly snapshot was taken, rounded to the hour. As the begin of the snapshot interval may vary for a few seconds or even minutes &mdash; between instances of the same database or different databases &mdash; this field helps to relate the snapshots.

<span style="font-style: italic; color: LightSlateGray; font-size:1.1em;">There are some lengty tests to show the snapshot intervals are identical and comparable. These tests are skipped here.</span>

In [8]:
# Sum up the statistics values of the two CUS DB Inte database instances
dtNulled.inte1 <- dtNulled.inte[, list("minInterval" = min(beginInterval), "maxInterval" = max(endInterval), "sumValue" = sum(offsetValue)),
   by = list(SNAP_ID, STAT_NAME, snapHour)]
   
# Same for the CUS DB Prod
dtNulled.prod1 <- dtNulled.prod[, list("minInterval" = min(beginInterval), "maxInterval" = max(endInterval), "sumValue" = sum(offsetValue)),
   by = list(SNAP_ID, STAT_NAME, snapHour)]

# Merge the data tables of CUS DB Prod and CUS DB Inte.The snapshots at August 29, 1:00 CEST and later are dropped because CUS DB Prod was restarted.
dtNulled <- merge(
   dtNulled.inte1[snapHour < as.POSIXct(strptime('2019-08-29 01:00:00', '%Y-%m-%d %H:%M:%S')), list(STAT_NAME, snapHour, sumValue)],
   dtNulled.prod1[snapHour < as.POSIXct(strptime('2019-08-29 01:00:00', '%Y-%m-%d %H:%M:%S')), list(STAT_NAME, snapHour, sumValue)],
   by=c("snapHour", "STAT_NAME"), all=T, suffixes=c("_I", "_P"), incomparables=NA)

setkeyv(dtNulled, c("snapHour", "STAT_NAME"))

# Rows with variable sumValue_I holds the statistics values of CUS DB Inte, and rows with variable sumValue_P for CUS DB Prod respectively
str(dtNulled)

Classes 'data.table' and 'data.frame':	114751 obs. of  4 variables:
 $ snapHour  : POSIXct, format: "2019-08-22 00:00:00" "2019-08-22 00:00:00" ...
 $ STAT_NAME : chr  "ADG parselock X get attempts" "ADG parselock X get successes" "Batched IO (bound) vector count" "Batched IO (full) vector count" ...
 $ sumValue_I: num  0 0 0 0 0 0 0 0 0 0 ...
 $ sumValue_P: num  0 0 0 0 0 0 0 0 0 0 ...
 - attr(*, ".internal.selfref")=<externalptr> 
 - attr(*, "sorted")= chr  "snapHour" "STAT_NAME"


### Save the workspace image with the cleansed data for subsequent analysis
This workspace will be imported for parts 2 and 3.

In [12]:
save(dtNulled, file="../analysis/R.workspaces/cleansed-data.RData")